In [1]:
from keras.layers import Input, Dense, Flatten
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19
import sys, glob, math, os
import cv2
import numpy as np
import re

# image sizes
img_rows, img_cols = 224, 224

Using TensorFlow backend.


In [2]:
def vgg19_lower_model():
    # initial model, load vgg19
    vgg19 = VGG19(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
    vgg19.summary()

    # new model that takes first 10 layers of VGG19
    model = Sequential()
    model.add(vgg19.get_layer('input_1'))
    model.add(vgg19.get_layer('block1_conv1'))
    model.add(vgg19.get_layer('block1_conv2'))
    model.add(vgg19.get_layer('block1_pool'))
    model.add(vgg19.get_layer('block2_conv1'))
    model.add(vgg19.get_layer('block2_conv2'))
    model.add(vgg19.get_layer('block2_pool'))
    model.add(vgg19.get_layer('block3_conv1'))
    model.add(vgg19.get_layer('block3_conv2'))
    model.add(vgg19.get_layer('block3_conv3'))
    model.add(vgg19.get_layer('block3_conv4'))
    model.add(vgg19.get_layer('block3_pool'))
    model.add(vgg19.get_layer('block4_conv1'))
    model.add(vgg19.get_layer('block4_conv2'))
    model.add(vgg19.get_layer('block4_conv3'))
    model.add(vgg19.get_layer('block4_conv4'))
    model.add(vgg19.get_layer('block4_pool'))
#    model.add(vgg19.get_layer('block5_conv1'))
#    model.add(vgg19.get_layer('block5_conv2'))
#    model.add(vgg19.get_layer('block5_conv3'))
#    model.add(vgg19.get_layer('block5_conv4'))
#    model.add(vgg19.get_layer('block5_pool'))
#    model.add(Flatten())
    model.summary()
 
    return model
#END vgg19_lower_model()

In [3]:
def load_imgs(dir,normalize=0):
    # load image data from dir

    data = []
    fname = []
    path  = dir + '/scene*.JPG'
    files = sorted(glob.glob(path))

    for fl in files:
        flbase = os.path.basename(fl)
        print('reading %s'%format(fl))

        img = cv2.imread(fl)
        img = cv2.resize(img, (img_rows,img_cols))
        img = np.array(img, dtype=np.float32)
        if normalize == 1:
            img -= np.mean(img)
            img /= np.std(img)

        data.append(img)
        fname.append(fl)

    data = np.array(data,dtype=np.float32)

    return data, fname
# end of load_imgs

In [4]:
def get_id_from_fname(str):
    
    m = re.match(r"([a-z/]+)([0-9]+)([a-z]+).[a-zA-Z]",str)

    if m:
    #    print("found!")
        id = int(m.group(2))
    #    print(id)
    else:
    #    print("not found!")
        id = -1
    
    return id
#end of get_id_from_frame

In [5]:
def save_array_csv(fname,val):

    f = open(fname, 'w')

    for i in range(0,val.shape[0]):
        for j in range(0,val.shape[1]):
            for k in range(0,val.shape[2]):
                f.write("%d, %d, %d, "%(i,j,k)) 
                print("%d, %d, %d, "%(i,j,k)) 
                for l in range(0,val.shape[3]):
                    f.write("%f, "%(val[i][j][k][l]))
                f.write("\n") 

    f.close()
#end of save_array_csv

In [6]:
if __name__=='__main__':

    data,fname = load_imgs('scenes',1) 
    print(data.shape)

reading scenes\scene007a.JPG
reading scenes\scene007b.JPG
reading scenes\scene007c.JPG
reading scenes\scene007d.JPG
reading scenes\scene007e.JPG
reading scenes\scene007f.JPG
reading scenes\scene007g.JPG
reading scenes\scene008a.JPG
reading scenes\scene008b.JPG
reading scenes\scene009a.JPG
reading scenes\scene009b.JPG
reading scenes\scene009c.JPG
reading scenes\scene009d.JPG
reading scenes\scene009e.JPG
reading scenes\scene009f.JPG
reading scenes\scene010a.JPG
reading scenes\scene010b.JPG
reading scenes\scene010c.JPG
reading scenes\scene010d.JPG
reading scenes\scene011a.JPG
reading scenes\scene011b.JPG
reading scenes\scene011c.JPG
reading scenes\scene012a.JPG
reading scenes\scene012b.JPG
reading scenes\scene012c.JPG
reading scenes\scene012d.JPG
reading scenes\scene013a.JPG
reading scenes\scene013b.JPG
reading scenes\scene013c.JPG
reading scenes\scene013d.JPG
reading scenes\scene014a.JPG
reading scenes\scene014b.JPG
reading scenes\scene014c.JPG
reading scenes\scene014d.JPG
reading scenes

In [7]:
    model = vgg19_lower_model()
    model.summary()
    out = model.predict(data, batch_size=32, verbose=1)

    print(out.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

201/201 [==============================] - 209s   
(201, 14, 14, 512)


In [8]:
#    save_array_csv("out.csv", out)
#    np.save("out.npy",out)

In [10]:
from scipy.cluster.vq import vq, kmeans, whiten
from sklearn.preprocessing import StandardScaler

data = np.reshape(out,(out.shape[0],out.shape[1]*out.shape[2],out.shape[3]))
print(data.shape)

(201, 196, 512)


In [11]:
# data standlization
data_for_cluster = np.reshape(data,(data.shape[0]*data.shape[1],data.shape[2]))

# standlization step
me = np.mean(data_for_cluster,0)
stv = np.std(data_for_cluster - me,0)
#stv2 = np.std(data_for_cluster,0)
#print(stv)
#print(stv2)
data_standlized = (data_for_cluster - me) / stv
data_whitened = data_standlized

In [12]:
# whiteneing step
#data_whitened = whiten(data_for_cluster)
#print(data_whitened)
#print(np.std(data_whitened,0))
centers, distortion = kmeans(data_whitened,200)
print(centers)
print(distortion)
print(data_whitened.shape)

[[-0.27162495 -0.13117421 -0.05720815 ...,  0.19951144 -0.22996446
   0.18999434]
 [ 0.41843849 -0.26691249 -0.13059689 ..., -0.1418881  -0.06818148
   0.43640643]
 [-0.16128561 -0.10454313  0.12421089 ...,  0.232299    0.361186   -0.1321549 ]
 ..., 
 [-0.1111643  -0.32594851 -0.2923643  ..., -0.1150239   0.31590223
  -0.29563385]
 [ 0.13345589 -0.20841429 -0.18622266 ...,  0.23623519 -0.00495818
   0.0884538 ]
 [-0.01612022 -0.29190692 -0.06765467 ..., -0.29334623  1.29137468
  -0.23022656]]
18.2183
(39396, 512)


In [12]:
#data = np.random.rand(100,5)
#standardized_data = StandardScaler().fit_transform(data)
#print(np.mean(standardized_data,0))

In [13]:
#np.save("data_whitened.npy",data_whitened)
np.save("data",data)
np.save("centers.npy",centers)
np.save("mean.npy",me)
np.save("stv.npy",stv)